In [1]:
import glob
import csv
import pandas as pd
import re

csv_path = ""
data_path = "../data/interim"
files = glob.glob(f"{data_path}/*uncalibrated.csv")
data_uncalibrated = [pd.read_csv(file, delimiter=',', index_col=0) for file in files]
classes = [int(re.search('stage_.+?', filename).group(0).split('_')[-1]) for filename in files]
print(len(classes), classes)
print(data_uncalibrated[0])
print(len(data_uncalibrated))

99 [5, 2, 2, 6, 5, 6, 1, 2, 2, 3, 6, 1, 5, 4, 2, 5, 6, 3, 5, 4, 3, 5, 2, 4, 3, 4, 6, 3, 1, 1, 5, 4, 2, 2, 4, 3, 5, 3, 4, 3, 5, 6, 3, 2, 3, 4, 1, 4, 1, 4, 2, 1, 2, 5, 4, 5, 5, 2, 3, 2, 3, 2, 4, 5, 5, 1, 1, 3, 2, 1, 1, 3, 5, 4, 1, 3, 2, 5, 1, 5, 6, 3, 6, 2, 3, 1, 4, 6, 1, 5, 3, 1, 4, 6, 2, 6, 5, 2, 4]
     X_UnCal    Y_UnCal  Z_UnCal     X_Bias    Y_Bias    Z_Bias  \
0     2.0740  24.558600 -64.4282 -29.610296  45.48195 -38.70174   
1    -0.0000  24.875800 -65.2212 -29.610298  45.48195 -38.70174   
2     2.3546  25.144200 -64.1842 -29.610298  45.48195 -38.70174   
3     1.9398  26.168999 -65.5384 -29.610298  45.48195 -38.70174   
4     1.4152  37.356400 -75.3594 -29.610298  45.48195 -38.70174   
..       ...        ...      ...        ...       ...       ...   
152   0.3538  55.680798 -54.8024 -29.610294  45.48195 -38.70174   
153  -1.9886  55.143997 -55.4734 -29.610294  45.48195 -38.70174   
154  -2.5376  55.058598 -56.8276 -29.610294  45.48195 -38.70174   
155  -1.6348  55.448997 -56.2

In [2]:
from scipy.fft import fft, fftfreq, rfft, rfftfreq
import numpy as np
def fourier(dataframe, start=5, end=200):
    yf_xu = rfft(dataframe['X_UnCal'].to_numpy())
    yf_yu = rfft(dataframe['Y_UnCal'].to_numpy())
    yf_zu = rfft(dataframe['Z_UnCal'].to_numpy())
    x = dataframe['time_uncalibrated'].to_numpy()
    x = x - x[0]
    xf = rfftfreq(len(x), (x[1] - x[0]) / 1000)
    return np.concatenate((yf_xu[start:end], yf_yu[start:end], yf_zu[start:end]))



In [3]:
X = [fourier(case, start=5, end=30) for case in data_uncalibrated]
# filter samples that are too small
index = [i  for i in range(len(X)) if len(X[i]) == 75]
X = [X[i] for i in index]
y = [classes[i] for i in index]

# to real values
X = np.real(X)
print(len(X))

77


In [5]:
from sklearn.model_selection import train_test_split
from sklearn import svm
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf = svm.SVC(decision_function_shape='ovo')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(sum([1 for i in range(len(y_pred)) if y_pred[i] == y_test[i]])/len(y_pred))
print(y_pred)
print(y_test)

0.038461538461538464
[1 1 1 2 1 2 2 1 2 1 1 2 3 1 1 1 1 2 1 1 1 3 1 1 1 1]
[5, 3, 4, 5, 3, 3, 2, 3, 5, 2, 4, 6, 2, 5, 5, 6, 5, 6, 2, 6, 5, 1, 4, 4, 6, 2]
